In [1]:
## Import Library ## 
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import PIL
import os
from tqdm import tqdm

pd.set_option("display.max_columns", None)

import warnings
warnings.filterwarnings("ignore")

In [ ]:
## Load Data ##
test = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')

for img_name in tqdm(test['image']):
    path = '../input/plant-pathology-2021-fgvc8/test_images/'+str(img_name)
    with PIL.Image.open(path) as img:
        img = img.resize((256,256))
        img.save(f'./{img_name}')

In [ ]:
## Data Generator ##
datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0,
                                                        preprocessing_function=None,
                                                        data_format=None,
                                                    )
datagen_aug = keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0,
                                                        preprocessing_function=None,
                                                        horizontal_flip = True,
                                                        vertical_flip = True,
                                                        data_format=None,
                                                    )

test_data = datagen.flow_from_dataframe(
    test,
    directory = './',
    x_col="image",
    y_col= None,
    color_mode="rgb",
    target_size = (256,256),
    classes=None,
    class_mode=None,
    batch_size=16,
    shuffle=False,
    seed=40,
)

test_data_aug = datagen_aug.flow_from_dataframe(
    test,
    directory = './',
    x_col="image",
    y_col= None,
    color_mode="rgb",
    target_size = (256,256),
    classes=None,
    class_mode=None,
    batch_size=16,
    shuffle=False,
    seed=40,
)

data = [test_data, test_data_aug]

In [4]:
## Model(NasnetLarge) ##
from keras.applications.nasnet import NASNetLarge
from keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
from keras.models import Model

def create_model_nas(input_shape, n_out):
    input_tensor = Input(shape = input_shape)
    base_model = NASNetLarge(weights = None, include_top = False, input_tensor = input_tensor)
    
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    final_output = Dense(n_out, activation='sigmoid', name='final_output')(x)
    model = Model(input_tensor, final_output)
    
    return model

In [5]:
## Model EfficientNetB7 ##
from keras.applications.efficientnet import EfficientNetB7
from keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
from keras.models import Model

def create_model_eff(input_shape, n_out):
    input_tensor = Input(shape = input_shape)
    base_model = EfficientNetB7(weights = None, include_top = False, input_tensor = input_tensor)
    
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    final_output = Dense(n_out, activation='sigmoid', name='final_output')(x)
    model = Model(input_tensor, final_output)
    
    return model

In [ ]:
model_eff = create_model_eff(input_shape=(256, 256, 3), n_out=6)
model_eff.load_weights('../input/d/minjuro/apple-leaf-checkpoint/ck/model.ckpt')

model_nas = create_model_nas(input_shape=(256, 256, 3), n_out=6)
model_nas.load_weights('../input/d/minjuro/apple-leaf-checkpoint/ck_nasnetlarge/model.ckpt')

models = [model_eff, model_nas]

In [ ]:
## EVAL ## 
preds = [model.predict(test) for model in models
                            for test in data]
preds = np.array(preds)
preds = np.mean(preds, axis=0)

In [8]:
preds = preds.tolist()

indices = []
for pred in preds:
    temp = []
    for category in pred:
        if category>=0.3:
            temp.append(pred.index(category))
    if temp!=[]:
        indices.append(temp)
    else:
        temp.append(np.argmax(pred))
        indices.append(temp)
    

In [9]:
labels = {'complex': 0, 'frog_eye_leaf_spot': 1, 'healthy': 2, 'powdery_mildew': 3, 'rust': 4, 'scab': 5}
labels = dict((v,k) for k,v in labels.items())

testlabels = []


for image in indices:
    temp = []
    for i in image:
        temp.append(str(labels[i]))
    testlabels.append(' '.join(temp))

In [10]:
delfiles = tf.io.gfile.glob('./*.jpg')

for file in delfiles:
    os.remove(file)

In [11]:
## Make Submission File ##
sub = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
sub['labels'] = testlabels
sub.to_csv('submission.csv', index=False)